In [57]:
import pandas as pd
import numpy as np
import h5py
import datetime

from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
np.random.seed(2017)

In [ ]:
def finetune_model(input_shape):
    input_tensor = Input(input_shape)
    x = Dropout(0.5)(input_tensor)
    x = Dense(1, activation='sigmoid')(x)
    model = Model(input_tensor, x)
    
    return model

    

In [5]:
batch_size = 128
epochs = 20

X_train = []
X_valid = []

file_features = [
    "features/finetune_ResNet50.h5",
    "features/finetune_Xception.h5",
    "features/finetune_InceptionV3.h5"
]

for filename in file_features:
    with h5py.File(filename, 'r') as h:
        X_train.append(np.array(h['train']))
        X_valid.append(np.array(h['valid']))
        y_train = np.array(h['label'])
        y_valid = np.array(h['valid_label'])

X_train = np.concatenate(X_train, axis=1)
X_valid = np.concatenate(X_valid, axis=1)

X_train, y_train = shuffle(X_train, y_train)
y_train = np.eye(10)[y_train]
X_valid, y_valid = shuffle(X_valid, y_valid)
y_valid = np.eye(10)[y_valid]

print(X_train.shape)
print(y_train.shape)

(20787, 6144)
(20787, 10)


In [47]:
def make_model(input_shape):

    input_tensor = Input(input_shape)
    x = input_tensor
    x = Dropout(0.5)(x)
    x = Dense(10, activation='softmax')(x)
    model = Model(input_tensor, x)
    
    return model

model_merged = make_model(X_train.shape[1:])


print("Adam")
model_merged.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_merged.fit(X_train, y_train, batch_size=batch_size, epochs=10, validation_data=(X_valid,y_valid))
model_merged.save("models/mixed-model.h5")
print("model save successed")

Adam
Train on 20787 samples, validate on 1637 samples
Epoch 1/10
20787/20787 [==============================] - 1s - loss: 0.1224 - acc: 0.9755 - val_loss: 0.6788 - val_acc: 0.7801
Epoch 2/10
20787/20787 [==============================] - 1s - loss: 0.0179 - acc: 0.9969 - val_loss: 0.6865 - val_acc: 0.7923
Epoch 3/10
20787/20787 [==============================] - 1s - loss: 0.0128 - acc: 0.9975 - val_loss: 0.6992 - val_acc: 0.7734
Epoch 4/10
20787/20787 [==============================] - 1s - loss: 0.0102 - acc: 0.9979 - val_loss: 0.7143 - val_acc: 0.7874
Epoch 5/10
20787/20787 [==============================] - 1s - loss: 0.0085 - acc: 0.9982 - val_loss: 0.7158 - val_acc: 0.7807
Epoch 6/10
20787/20787 [==============================] - 1s - loss: 0.0068 - acc: 0.9986 - val_loss: 0.7524 - val_acc: 0.7746
Epoch 7/10
20787/20787 [==============================] - 1s - loss: 0.0065 - acc: 0.9985 - val_loss: 0.7957 - val_acc: 0.7740
Epoch 8/10
20787/20787 [==============================] -

In [62]:
def make_predictions(MODEL, X_test, image_size, batch_size):
    y_pred = MODEL.predict(X_test, verbose=1)
    y_pred = y_pred.clip(min=0.005, max=0.995)
    
    gen = ImageDataGenerator()
    path_test_data = 'dataset/to_prediction'
    test_generator = gen.flow_from_directory(path_test_data,image_size, shuffle=False, 
                                             batch_size=batch_size, class_mode=None)
    
    test_id = list()
    for i, file_name in enumerate(test_generator.filenames):
        flbase = os.path.basename(file_name)
        test_id.append(flbase)        
    
    return y_pred, test_id

def create_submission(predictions, test_id):
    result1 = pd.DataFrame(predictions, columns=['c0', 'c1', 'c2', 'c3',
                                                 'c4', 'c5', 'c6', 'c7',
                                                 'c8', 'c9'])
    result1.loc[:, 'img'] = pd.Series(test_id, index=result1.index)
    now = datetime.datetime.now()
    if not os.path.isdir('subm'):
        os.mkdir('subm')
    suffix = str(now.strftime("%Y-%m-%d-%H-%M"))
    sub_file = os.path.join('subm', 'submission_' + suffix + '.csv')
    result1.to_csv(sub_file, index=False)

In [60]:
test_features = [
     "features/finetune_test_ResNet50.h5",
    "features/finetune_test_Xception.h5",
    "features/finetune_test_InceptionV3.h5"
]
X_test = []
for filename in test_features:
    with h5py.File(filename, 'r') as h:
        X_test.append(np.array(h['test']))
        
X_test = np.concatenate(X_test, axis=1)
predictions, test_id = make_predictions(model_merged, X_test, (299, 299), batch_size)

79392/79726 [============================>.] - ETA: 0sFound 79726 images belonging to 1 classes.


In [61]:
create_submission(predictions, test_id)